...

In [1]:
from torchvision import datasets, transforms
import torch
from torch import nn
from torch.utils.data import DataLoader

from sklearn.manifold import TSNE
import pandas as pd
import seaborn as sns

In [3]:
batch_size = 1
device = "cuda"
print(torch.cuda.is_available())

True


#### Instanciamos la red para generar los embeddings

In [4]:

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        ) 

    def forward(self, x):
        x = self.flatten(x)
        embedding = self.linear_relu_stack[:-1](x)  
        logits = self.linear_relu_stack(x)
        return embedding, logits

In [19]:
# [(num+15, train_data[i][1]) for num, i in enumerate(range(15, 40))]

#### Generamos el clasificador con los pesos malos

In [5]:
from pprint import pprint
estado_fase_0 = torch.load('Fase_0.pth')
estado_fase_1 = torch.load('Fase_1.pth')

estado_fase_1["linear_relu_stack.4.weight"][5:10, :] = estado_fase_0["linear_relu_stack.4.weight"][5:10, :]
estado_fase_1["linear_relu_stack.4.bias"][5:10] = estado_fase_0["linear_relu_stack.4.bias"][5:10]

In [6]:
print("Weights Norm:")
wn = [(indice ,torch.norm(row) ) for indice ,row in enumerate(estado_fase_1["linear_relu_stack.4.weight"])]
pprint(wn)
print("Bias:")
bias = [(indice ,row ) for indice ,row in enumerate(estado_fase_1["linear_relu_stack.4.bias"])]
pprint(bias)

Weights Norm:
[(0, tensor(1.0966, device='cuda:0')),
 (1, tensor(1.1971, device='cuda:0')),
 (2, tensor(1.0860, device='cuda:0')),
 (3, tensor(1.0435, device='cuda:0')),
 (4, tensor(1.0837, device='cuda:0')),
 (5, tensor(0.6537, device='cuda:0')),
 (6, tensor(0.6658, device='cuda:0')),
 (7, tensor(0.6536, device='cuda:0')),
 (8, tensor(0.6560, device='cuda:0')),
 (9, tensor(0.6453, device='cuda:0'))]
Bias:
[(0, tensor(-0.0329, device='cuda:0')),
 (1, tensor(0.2524, device='cuda:0')),
 (2, tensor(0.0166, device='cuda:0')),
 (3, tensor(0.0098, device='cuda:0')),
 (4, tensor(0.0994, device='cuda:0')),
 (5, tensor(-0.0808, device='cuda:0')),
 (6, tensor(-0.1150, device='cuda:0')),
 (7, tensor(-0.0970, device='cuda:0')),
 (8, tensor(-0.1229, device='cuda:0')),
 (9, tensor(-0.0929, device='cuda:0'))]


In [10]:
model = NeuralNetwork().to(device)
model.load_state_dict(estado_fase_1)

<All keys matched successfully>

#### Elegimos una de las clases

In [7]:

from torch.utils.data import DataLoader
import torchvision
from torchvision import transforms

eval_data = torchvision.datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
# Conjunto de evaluación
eval_indices_to_7 = [i for i in range(len(eval_data)) if eval_data.targets[i] == 7]

eval_to_7 = torch.utils.data.Subset(eval_data, eval_indices_to_7)
eval_to_7_dataloader = DataLoader(eval_to_7, batch_size, shuffle=True)

In [8]:
def extractor(dataloader, model):
    model.eval()
    embeddings = list()
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            features, logist_ = model(X)
            embeddings.append( (features.cpu(), logist_.cpu(),y.cpu()) )
            
    return embeddings

#### Aquí tenemos todos los embeddings de todas las imagenes del número 7

In [11]:
embeddings =  extractor(eval_to_7_dataloader, model)

In [24]:
# Embedding
embeddings[0][0].shape

torch.Size([1, 512])

In [21]:
select = 19
print(torch.norm(embeddings[select][0]) ,embeddings[select][2],torch.argmax(embeddings[select][1]))

tensor(14.8855) tensor([7]) tensor(7)


#### Volvamos a la matriz de pesos

In [23]:
estado_fase_1["linear_relu_stack.4.weight"].shape

torch.Size([10, 512])

- Queremos encontrar los 100 indices con los productos mas altos

In [49]:
vector = embeddings[0][0] # torch.Size([1, 512])
weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])

In [69]:
def calculate_dot_products(vector, weights):
    dot_products_manual = []
    max_indices = []

    # Realizar el producto punto manualmente y guardar el índice del mayor producto
    for i, row in enumerate(weights):
        max_product = float("-inf")
        max_index = None
        dot_product = 0
        for j in range(len(vector[0])):
            dot_product += row[j] * vector[0][j]
            if row[j] * vector[0][j] > max_product:
                max_product = row[j] * vector[0][j]
                max_index = j
        dot_products_manual.append(dot_product)
        max_indices.append((i, max_index))

    # Mostrar los números y sus índices
    for i, tensor in enumerate(dot_products_manual):
        print("Índice:", i, "Producto punto:", tensor.item())
    print("Indice con el mayor producto")
    print( max_indices)

In [70]:
vector = embeddings[0][0] # torch.Size([1, 512])
weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
calculate_dot_products(vector, weights)

Índice: 0 Producto punto: -4.719912052154541
Índice: 1 Producto punto: -2.4129674434661865
Índice: 2 Producto punto: -4.8340535163879395
Índice: 3 Producto punto: -2.32509708404541
Índice: 4 Producto punto: -0.7826246619224548
Índice: 5 Producto punto: -1.5168896913528442
Índice: 6 Producto punto: -1.4903335571289062
Índice: 7 Producto punto: 0.47635093331336975
Índice: 8 Producto punto: -0.8869917988777161
Índice: 9 Producto punto: -0.9040484428405762
Indice con el mayor producto
[(0, 325), (1, 288), (2, 388), (3, 439), (4, 409), (5, 277), (6, 114), (7, 188), (8, 151), (9, 47)]


In [71]:
vector = embeddings[1][0] # torch.Size([1, 512])
weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
calculate_dot_products(vector, weights)

Índice: 0 Producto punto: -4.951585292816162
Índice: 1 Producto punto: -3.768437147140503
Índice: 2 Producto punto: -6.010313034057617
Índice: 3 Producto punto: -2.6406893730163574
Índice: 4 Producto punto: -1.450272798538208
Índice: 5 Producto punto: -1.795319676399231
Índice: 6 Producto punto: -1.8316062688827515
Índice: 7 Producto punto: 0.6995322108268738
Índice: 8 Producto punto: -1.0334159135818481
Índice: 9 Producto punto: -1.172971248626709
Indice con el mayor producto
[(0, 325), (1, 288), (2, 132), (3, 439), (4, 409), (5, 277), (6, 114), (7, 188), (8, 151), (9, 47)]


In [72]:
vector = embeddings[2][0] # torch.Size([1, 512])
weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
calculate_dot_products(vector, weights)

Índice: 0 Producto punto: -7.391018867492676
Índice: 1 Producto punto: -5.288793563842773
Índice: 2 Producto punto: -8.529760360717773
Índice: 3 Producto punto: -3.5293660163879395
Índice: 4 Producto punto: -2.2228734493255615
Índice: 5 Producto punto: -2.6832709312438965
Índice: 6 Producto punto: -2.6769754886627197
Índice: 7 Producto punto: 0.9215599894523621
Índice: 8 Producto punto: -1.2699629068374634
Índice: 9 Producto punto: -1.7799588441848755
Indice con el mayor producto
[(0, 325), (1, 288), (2, 132), (3, 439), (4, 409), (5, 277), (6, 114), (7, 188), (8, 151), (9, 47)]


In [73]:
vector = embeddings[3][0] # torch.Size([1, 512])
weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
calculate_dot_products(vector, weights)

Índice: 0 Producto punto: -3.9612576961517334
Índice: 1 Producto punto: -6.551476955413818
Índice: 2 Producto punto: -8.131542205810547
Índice: 3 Producto punto: -4.041262149810791
Índice: 4 Producto punto: -1.9478480815887451
Índice: 5 Producto punto: -1.6651408672332764
Índice: 6 Producto punto: -1.721736192703247
Índice: 7 Producto punto: -0.3020082116127014
Índice: 8 Producto punto: -1.4926122426986694
Índice: 9 Producto punto: -1.212050437927246
Indice con el mayor producto
[(0, 325), (1, 288), (2, 312), (3, 53), (4, 409), (5, 277), (6, 114), (7, 188), (8, 409), (9, 104)]


In [74]:
vector = embeddings[4][0] # torch.Size([1, 512])
weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
calculate_dot_products(vector, weights)

Índice: 0 Producto punto: -4.027512550354004
Índice: 1 Producto punto: -4.163403511047363
Índice: 2 Producto punto: -5.442128658294678
Índice: 3 Producto punto: -2.066265106201172
Índice: 4 Producto punto: -0.668584406375885
Índice: 5 Producto punto: -1.5021089315414429
Índice: 6 Producto punto: -1.5751320123672485
Índice: 7 Producto punto: 0.4001258909702301
Índice: 8 Producto punto: -0.9389763474464417
Índice: 9 Producto punto: -0.98646479845047
Indice con el mayor producto
[(0, 325), (1, 288), (2, 132), (3, 439), (4, 409), (5, 277), (6, 114), (7, 188), (8, 104), (9, 26)]


In [75]:
vector = embeddings[5][0] # torch.Size([1, 512])
weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
calculate_dot_products(vector, weights)

Índice: 0 Producto punto: -5.842214107513428
Índice: 1 Producto punto: -6.18215274810791
Índice: 2 Producto punto: -7.770198345184326
Índice: 3 Producto punto: -3.825814962387085
Índice: 4 Producto punto: -2.8593456745147705
Índice: 5 Producto punto: -2.6373395919799805
Índice: 6 Producto punto: -2.5506114959716797
Índice: 7 Producto punto: 1.033263921737671
Índice: 8 Producto punto: -1.3971160650253296
Índice: 9 Producto punto: -1.9435690641403198
Indice con el mayor producto
[(0, 325), (1, 288), (2, 132), (3, 439), (4, 409), (5, 277), (6, 114), (7, 188), (8, 151), (9, 104)]


In [80]:
for i in range(15, 17):
    vector = embeddings[i][0] # torch.Size([1, 512])
    weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
    calculate_dot_products(vector, weights)

Índice: 0 Producto punto: -8.379469871520996
Índice: 1 Producto punto: -2.701742649078369
Índice: 2 Producto punto: -6.433638572692871
Índice: 3 Producto punto: -2.3344814777374268
Índice: 4 Producto punto: -3.376556634902954
Índice: 5 Producto punto: -2.4215102195739746
Índice: 6 Producto punto: -2.1800248622894287
Índice: 7 Producto punto: -0.267403781414032
Índice: 8 Producto punto: -1.0816799402236938
Índice: 9 Producto punto: -1.427729606628418
Indice con el mayor producto
[(0, 325), (1, 327), (2, 388), (3, 53), (4, 409), (5, 277), (6, 114), (7, 188), (8, 151), (9, 47)]
Índice: 0 Producto punto: -3.3843958377838135
Índice: 1 Producto punto: -2.3225581645965576
Índice: 2 Producto punto: -3.8065364360809326
Índice: 3 Producto punto: -1.633362054824829
Índice: 4 Producto punto: -0.34295904636383057
Índice: 5 Producto punto: -1.146543264389038
Índice: 6 Producto punto: -1.1166719198226929
Índice: 7 Producto punto: 0.3229431211948395
Índice: 8 Producto punto: -0.7552940845489502
Índice

### Veamos que pasa con otra clase

In [81]:
eval_data = torchvision.datasets.MNIST(
    root="../data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)
# Conjunto de evaluación
eval_indices_to_3 = [i for i in range(len(eval_data)) if eval_data.targets[i] == 3]

eval_to_3 = torch.utils.data.Subset(eval_data, eval_indices_to_3)
eval_to_3_dataloader = DataLoader(eval_to_3, batch_size, shuffle=True)

In [82]:
embeddings_3 =  extractor(eval_to_3_dataloader, model)

In [93]:
for select in range(20):

    print(select,torch.norm(embeddings_3[select][0]) ,embeddings_3[select][2],torch.argmax(embeddings_3[select][1]))

0 tensor(20.2078) tensor([3]) tensor(3)
1 tensor(17.7633) tensor([3]) tensor(8)
2 tensor(20.9028) tensor([3]) tensor(8)
3 tensor(9.2237) tensor([3]) tensor(6)
4 tensor(22.1079) tensor([3]) tensor(3)
5 tensor(17.7273) tensor([3]) tensor(3)
6 tensor(13.5157) tensor([3]) tensor(3)
7 tensor(12.1855) tensor([3]) tensor(3)
8 tensor(19.1539) tensor([3]) tensor(3)
9 tensor(11.7066) tensor([3]) tensor(1)
10 tensor(14.1310) tensor([3]) tensor(3)
11 tensor(9.2363) tensor([3]) tensor(3)
12 tensor(14.8684) tensor([3]) tensor(3)
13 tensor(14.2826) tensor([3]) tensor(3)
14 tensor(10.6723) tensor([3]) tensor(3)
15 tensor(16.6160) tensor([3]) tensor(8)
16 tensor(17.6403) tensor([3]) tensor(3)
17 tensor(17.5229) tensor([3]) tensor(3)
18 tensor(9.9987) tensor([3]) tensor(6)
19 tensor(18.7090) tensor([3]) tensor(3)


In [94]:
for i in range(9, 11):
    print(i)
    vector = embeddings_3[i][0] # torch.Size([1, 512])
    weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
    calculate_dot_products(vector, weights)

9
Índice: 0 Producto punto: -6.3605804443359375
Índice: 1 Producto punto: -0.5502133369445801
Índice: 2 Producto punto: -2.3556411266326904
Índice: 3 Producto punto: -0.6394689679145813
Índice: 4 Producto punto: -4.398438453674316
Índice: 5 Producto punto: -1.6422111988067627
Índice: 6 Producto punto: -1.3337544202804565
Índice: 7 Producto punto: -1.5603241920471191
Índice: 8 Producto punto: -0.7158467769622803
Índice: 9 Producto punto: -1.1924173831939697
Indice con el mayor producto
[(0, 325), (1, 444), (2, 388), (3, 496), (4, 409), (5, 277), (6, 114), (7, 47), (8, 277), (9, 47)]
10
Índice: 0 Producto punto: -3.9866647720336914
Índice: 1 Producto punto: -4.364236354827881
Índice: 2 Producto punto: -4.690904140472412
Índice: 3 Producto punto: 0.9349015951156616
Índice: 4 Producto punto: -6.287780284881592
Índice: 5 Producto punto: -1.3395602703094482
Índice: 6 Producto punto: -1.3677284717559814
Índice: 7 Producto punto: -2.181875705718994
Índice: 8 Producto punto: -1.1456067562103271

In [95]:
for i in range(18, 20):
    print(i)
    vector = embeddings_3[i][0] # torch.Size([1, 512])
    weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
    calculate_dot_products(vector, weights)

18
Índice: 0 Producto punto: -3.995173692703247
Índice: 1 Producto punto: -2.126204252243042
Índice: 2 Producto punto: -2.4502220153808594
Índice: 3 Producto punto: -1.1980994939804077
Índice: 4 Producto punto: -3.9743621349334717
Índice: 5 Producto punto: -1.1503819227218628
Índice: 6 Producto punto: -0.6921254992485046
Índice: 7 Producto punto: -1.6696603298187256
Índice: 8 Producto punto: -0.8027104735374451
Índice: 9 Producto punto: -0.980441689491272
Indice con el mayor producto
[(0, 325), (1, 444), (2, 388), (3, 496), (4, 409), (5, 277), (6, 114), (7, 114), (8, 277), (9, 277)]
19
Índice: 0 Producto punto: -4.612472057342529
Índice: 1 Producto punto: -5.0663371086120605
Índice: 2 Producto punto: -5.688878536224365
Índice: 3 Producto punto: 0.9830247759819031
Índice: 4 Producto punto: -9.658469200134277
Índice: 5 Producto punto: -1.8969740867614746
Índice: 6 Producto punto: -1.81869375705719
Índice: 7 Producto punto: -2.798523426055908
Índice: 8 Producto punto: -1.546491026878357
Í

In [97]:
for i in range(14, 16):
    print(i)
    vector = embeddings_3[i][0] # torch.Size([1, 512])
    weights = estado_fase_1["linear_relu_stack.4.weight"] # torch.Size([10, 512])
    calculate_dot_products(vector, weights)

14
Índice: 0 Producto punto: -1.8345768451690674
Índice: 1 Producto punto: -3.4103941917419434
Índice: 2 Producto punto: -3.1460728645324707
Índice: 3 Producto punto: 0.7246735095977783
Índice: 4 Producto punto: -5.723138332366943
Índice: 5 Producto punto: -1.0889941453933716
Índice: 6 Producto punto: -1.0627979040145874
Índice: 7 Producto punto: -1.5920348167419434
Índice: 8 Producto punto: -1.0199458599090576
Índice: 9 Producto punto: -1.4818040132522583
Indice con el mayor producto
[(0, 325), (1, 444), (2, 199), (3, 184), (4, 409), (5, 277), (6, 114), (7, 114), (8, 277), (9, 277)]
15
Índice: 0 Producto punto: -8.016717910766602
Índice: 1 Producto punto: -2.724132537841797
Índice: 2 Producto punto: -4.613162517547607
Índice: 3 Producto punto: -1.9271371364593506
Índice: 4 Producto punto: -5.372285842895508
Índice: 5 Producto punto: -2.125352144241333
Índice: 6 Producto punto: -1.6529182195663452
Índice: 7 Producto punto: -1.9010593891143799
Índice: 8 Producto punto: -0.83078771829605